In [2]:
import sys
import torch
import os
from os import makedirs
import numpy as np
import open3d as o3d
from random import randint
from scipy.spatial.transform import Rotation
from tqdm import tqdm
import matplotlib.pyplot as plt

np.set_printoptions(suppress=True)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
def getWorld2View2(R, t, translate=np.array([.0, .0, .0]), scale=1.0):
    Rt = np.zeros((4, 4))
    Rt[:3, :3] = R.transpose()
    Rt[:3, 3] = t
    Rt[3, 3] = 1.0

    C2W = np.linalg.inv(Rt)
    cam_center = C2W[:3, 3]
    cam_center = (cam_center + translate) * scale
    C2W[:3, 3] = cam_center
    Rt = np.linalg.inv(C2W)
    return np.float32(Rt)

def qvec2rotmat(qvec):
    return np.array([
        [1 - 2 * qvec[2]**2 - 2 * qvec[3]**2,
         2 * qvec[1] * qvec[2] - 2 * qvec[0] * qvec[3],
         2 * qvec[3] * qvec[1] + 2 * qvec[0] * qvec[2]],
        [2 * qvec[1] * qvec[2] + 2 * qvec[0] * qvec[3],
         1 - 2 * qvec[1]**2 - 2 * qvec[3]**2,
         2 * qvec[2] * qvec[3] - 2 * qvec[0] * qvec[1]],
        [2 * qvec[3] * qvec[1] - 2 * qvec[0] * qvec[2],
         2 * qvec[2] * qvec[3] + 2 * qvec[0] * qvec[1],
         1 - 2 * qvec[1]**2 - 2 * qvec[2]**2]])

In [4]:
# Camera Intrinsics
rgb_camera_params = {}
depth_camera_params = {}
relative_positions = {}

def readRGBDConfig(config_file):
    # Define dictionaries to hold camera parameters
    # rgb_camera_params = {}
    # depth_camera_params = {}
    # relative_positions = {}
    
    with open(config_file, 'r') as file:
        data = file.read().split('\n\n')
    
        # Read RGB camera parameters
        rgb_data = data[0].split('\n')
        for line in rgb_data[1:4]:
            key, value = line.split('=')
            if ',' in value:
                value = tuple(map(float, value.split(',')))
            else:
                value = tuple(map(int, value.split('x')))
            rgb_camera_params[key] = value

        vFOV, hFOV = rgb_data[4].split(',')
        key, value = vFOV.split('=')
        rgb_camera_params[key] = float(value.strip('°'))
        key, value = hFOV.split('=')
        rgb_camera_params[key.strip(' ')] = float(value.strip('°'))

        # Read Depth camera parameters
        depth_data = data[1].split('\n')
        for line in depth_data[1:4]:
            key, value = line.split('=')
            if ',' in value:
                value = tuple(map(float, value.split(',')))
            else:
                value = tuple(map(int, value.split('x')))
            depth_camera_params[key] = value

        vFOV, hFOV = depth_data[4].split(',')
        key, value = vFOV.split('=')
        depth_camera_params[key] = float(value.strip('°'))
        key, value = hFOV.split('=')
        depth_camera_params[key.strip(' ')] = float(value.strip('°'))

    
        # Read relative positions of camera components
        rel_pos_data = data[2].split('\n')
        for line in rel_pos_data[1:]:
            key, value = line.split(': ')
            value = tuple(map(float, value.strip('(').strip(')').split(',')))
            relative_positions[key] = value

        # return rgb_camera_params, depth_camera_params, relative_positions
    
    # Access the loaded camera parameters
    print("RGB Camera Parameters:")
    print(rgb_camera_params)
    
    print("\nDepth Camera Parameters:")
    print(depth_camera_params)
    
    print("\nRelative Positions of Camera Components:")
    print(relative_positions)

conifg = "G:\\Universitat Siegen\\SA\\P-GPU\\Code\\gaussian-splatting\\data\\RGBD_Data\\config\\configuration.txt"
readRGBDConfig(conifg)

RGB Camera Parameters:
{'resolution': (1920, 1080), 'cx,cy,fx,fy': (959.5, 539.5, 960.0, 960.0), 'k1,k2,k3,p1,p2': (0.0, 0.0, 0.0, 0.0, 0.0), 'vFov': 58.7155, 'hFov': 90.0}

Depth Camera Parameters:
{'resolution': (512, 424), 'cx,cy,fx,fy': (255.5, 211.5, 256.0, 256.0), 'k1,k2,k3,p1,p2': (0.0, 0.0, 0.0, 0.0, 0.0), 'vFov': 79.2579, 'hFov': 89.9999}

Relative Positions of Camera Components:
{'RGB sensor': (0.0, 0.0, 0.0), 'RGB light source': (0.0, 0.3, 0.0), 'RGB light source size': (0.4, 0.4), 'Depth sensor': (0.03, 0.0, 0.0), 'Depth light source': (0.06, 0.0, 0.0), 'Depth light source size': (0.02, 0.02)}


In [5]:
# Camera Extrinsics
directory = "G:\\Universitat Siegen\\SA\\P-GPU\\Code\\gaussian-splatting\\data\\RGBD_Data\\config"

config_files = os.listdir(directory)
config_files.sort()

extrinsics = {}

# config_files[:5]
# for file in tqdm(range(len(config_files))):
for config_index in tqdm(range(5)):
    file = config_files[config_index]
    if file.startswith("campose-rgb-"):
        frame_id = file.split('-')[2].split('.')[0]
        config_file = os.path.join(directory, file)
        print(config_file)
        
        with open(config_file, 'r') as file:
            lines = file.readlines()

            # Extracting position
            position_str = lines[0].replace('position=', '').split('\n')[0]
            position = np.array([float(i) for i in position_str.strip('()').split(',')])

            # Extracting rotation as a quaternion
            rotation_str = lines[1].replace('rotation_as_quaternion=', '').split('\n')[0]
            rotation = np.array([float(i) for i in rotation_str.strip('()').split(',')])

            # Extracting the 4x4 pose matrix
            pose_str = lines[3:]
            pose = np.array([[float(i) for i in row.strip('(').split(')')[0].split(',')] for row in pose_str if row != ''])    

            # print('Position:', position)
            # print('Rotation:',rotation)
            # print('Pose:',pose)
            extrinsics[frame_id] = pose

100%|██████████| 5/5 [00:00<00:00, 5005.14it/s]

G:\Universitat Siegen\SA\P-GPU\Code\gaussian-splatting\data\RGBD_Data\config\campose-rgb-0000.txt
G:\Universitat Siegen\SA\P-GPU\Code\gaussian-splatting\data\RGBD_Data\config\campose-rgb-0001.txt
G:\Universitat Siegen\SA\P-GPU\Code\gaussian-splatting\data\RGBD_Data\config\campose-rgb-0002.txt
G:\Universitat Siegen\SA\P-GPU\Code\gaussian-splatting\data\RGBD_Data\config\campose-rgb-0003.txt
G:\Universitat Siegen\SA\P-GPU\Code\gaussian-splatting\data\RGBD_Data\config\campose-rgb-0004.txt


In [6]:
# Preprocess RGBD to PLY


# Directory where your images are stored
directory = "G:\\Universitat Siegen\\SA\\P-GPU\\Code\\gaussian-splatting\\data\\RGBD_Data\\rgb"
ply_path = "G:\\Universitat Siegen\\SA\\P-GPU\\Code\\gaussian-splatting\\data\\RGBD_Data\\ply"

# Get the list of files in the directory
files = os.listdir(directory)

# Sort the files to process depth and color images together
files.sort()

# Initialize an empty point cloud
point_cloud = o3d.geometry.PointCloud()

start_index = 0
ply_files = os.listdir(ply_path)
if len(ply_files) > 0:
    start_index = int(ply_files[-1].split('.')[0])

# Loop through each pair of depth and color images
# for i in tqdm(range(start_index, len(files))):
for i in tqdm(range(1)):
    file = files[i]
    # print(file)
    if file.startswith("gt-rgb-depth-"):  # Check if the file is a depth image
        depth_file = os.path.join(directory, file)
        
        # Get the corresponding color image
        color_file = os.path.join(directory, "rgb-" + file[-8:])  # Assuming both files have corresponding indices
        
        frame_id = file.split('-')[3].split('.')[0]

        # Read the depth and color images
        depth_image = o3d.io.read_image(depth_file)
        color_image = o3d.io.read_image(color_file)
        
        # Convert images to numpy arrays
        depth_array = np.asarray(depth_image)
        color_array = np.asarray(color_image)

        width = depth_array.shape[1]
        height = depth_array.shape[0]
        
        # Intrinsic parameters of the camera (you may need to adjust these values)
        intrinsic = o3d.camera.PinholeCameraIntrinsic()
        cx = rgb_camera_params['cx,cy,fx,fy'][0]
        cy = rgb_camera_params['cx,cy,fx,fy'][1]
        fx = rgb_camera_params['cx,cy,fx,fy'][2] 
        fy = rgb_camera_params['cx,cy,fx,fy'][3]
        intrinsic.set_intrinsics(width=width, height=height, cx=cx, cy=cy, fx=fx, fy=fy)

        # print("I:", intrinsic.intrinsic_matrix)
        # print("P:", extrinsics[frame_id])

        assert frame_id in extrinsics, "No pose found for frame " + frame_id

        pose = extrinsics[frame_id]
        # Extraction of rotation matrix (3x3 sub-matrix)
        rotation_matrix = pose[:3, :3]

        # Extraction of translation vector (last column)
        translation_vector = pose[:3, 3]

        # print(pose)
        # worldtoview = getWorld2View2(rotation_matrix, translation_vector)
        print(rotation_matrix)
        # print(translation_vector)
        # print("W2V:", worldtoview)

        # Create a point cloud from the depth and color information
        rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(color_image, depth_image, depth_trunc=4.0, convert_rgb_to_intensity=False)
        pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic,extrinsic=extrinsics[frame_id])
        
        pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
        
        # Save the point cloud to a file
        ply_file_path = os.path.join(ply_path, frame_id + ".ply")    
        
        o3d.io.write_point_cloud(ply_file_path, pcd)

        # Clear memory
        depth_image = None
        color_image = None
        depth_array = None
        color_array = None
        pcd = None

        # Merge current point cloud with the overall point cloud
        # point_cloud += pcd

# Visualize the final point cloud
# o3d.visualization.draw_geometries([point_cloud])

  0%|          | 0/1 [00:00<?, ?it/s]

[[-1.          0.00000004 -0.00000008]
 [-0.          0.866025    0.5       ]
 [ 0.00000009  0.5        -0.866025  ]]


100%|██████████| 1/1 [00:02<00:00,  2.15s/it]


In [7]:
rot = qvec2rotmat([1.13133e-08,0.965926,0.258819,-4.2222e-08])
rot2 = Rotation.from_quat([1.13133e-08,0.965926,0.258819,-4.2222e-08]).as_matrix()

print(rot)
print('')
print(rot2)

[[ 0.86602545  0.5        -0.00000008]
 [ 0.5        -0.86602607 -0.00000004]
 [-0.00000009 -0.         -1.00000062]]

[[-1.          0.00000004 -0.00000008]
 [-0.          0.86602549  0.49999985]
 [ 0.00000009  0.49999985 -0.86602549]]


In [22]:
# Visualize the final point cloud
ply_file_path = os.path.join(ply_path, '0000_P.ply')
ply_file_path = os.path.join(ply_path, 'Global.ply')

# 4X4 transformation matrix
transformation_matrix =  [[1, 0, 0, 0.5],
                            [0, 1, 0, 0],
                            [0, 0, 1, 0],
                            [0, 0, 0, 1]]


ply_files = os.listdir(ply_path)
pcd_list = []
for file in ply_files[:1]:
    if file.endswith(".ply"):
        pcd_file = os.path.join(ply_path, file)
        pcd = o3d.io.read_point_cloud(pcd_file)
        pcd.transform(transformation_matrix)
        pcd_list.append(pcd)

pcd_list.append(o3d.io.read_point_cloud(ply_file_path))
o3d.visualization.draw_geometries(pcd_list)
